<h1 align="center">Dataframe creation</h1>

In [39]:
import pandas as pd

# Load DataFrames
dim_campaigns = pd.read_csv('D:/codebasics c9/C9_Input_Files/C9_Input_Files/dim_campaigns.csv')
dim_products = pd.read_csv('D:/codebasics c9/C9_Input_Files/C9_Input_Files/dim_products.csv')
dim_stores = pd.read_csv('D:/codebasics c9/C9_Input_Files/C9_Input_Files/dim_stores.csv')
fact_events = pd.read_csv('D:/codebasics c9/C9_Input_Files/C9_Input_Files/fact_events.csv')



<h1 align="center">Data modeling as per fact and dimension tables
</h1>

In [40]:
# Merge DataFrames
merged_df = pd.merge(fact_events, dim_campaigns, on='campaign_id')
merged_df = pd.merge(merged_df, dim_products, on='product_code')
merged_df = pd.merge(merged_df, dim_stores, on='store_id')




In [41]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   event_id                     1500 non-null   object
 1   store_id                     1500 non-null   object
 2   campaign_id                  1500 non-null   object
 3   product_code                 1500 non-null   object
 4   base_price                   1500 non-null   int64 
 5   promo_type                   1500 non-null   object
 6   quantity_sold(before_promo)  1500 non-null   int64 
 7   quantity_sold(after_promo)   1500 non-null   int64 
 8   campaign_name                1500 non-null   object
 9   start_date                   1500 non-null   object
 10  end_date                     1500 non-null   object
 11  product_name                 1500 non-null   object
 12  category                     1500 non-null   object
 13  city                         1500

In [52]:
merged_df.to_csv('D:/codebasics c9/C9_Input_Files/C9_Input_Files/merged_df.csv', index=False)

<h1 align="center">Calculated Column Creation as per business logic 
</h1>

In [42]:
# Add necessary columns
merged_df['promotional_price'] = merged_df.apply(lambda row: 
    row['base_price'] * 0.5 if row['promo_type'] == 'BOGOF' else
    row['base_price'] * 0.75 if row['promo_type'] == '25% OFF' else
    row['base_price'] * 0.5 if row['promo_type'] == '50% OFF' else
    row['base_price'] * 0.67 if row['promo_type'] == '33% OFF' else
    row['base_price'] - 500 if row['promo_type'] == '500 Cashback' else
    row['base_price'],
    axis=1
)

merged_df['new_quantity'] = merged_df.apply(lambda row: 
    row['quantity_sold(after_promo)'] * 2 if row['promo_type'] == 'BOGOF' else
    row['quantity_sold(after_promo)'],
    axis=1
)

merged_df['revenue_before_promotion'] = merged_df['base_price'] * merged_df['quantity_sold(before_promo)']
merged_df['revenue_after_promotion'] = merged_df['new_quantity'] * merged_df['promotional_price']



<h1 align="center">Two kpi column IR , ISU Creation</h1>

In [43]:
# Calculate ISU and IR
merged_df['iSU'] = merged_df['new_quantity'] - merged_df['quantity_sold(before_promo)']
merged_df['ir'] = merged_df['revenue_after_promotion'] - merged_df['revenue_before_promotion']



<h1 align="center">Exploratory Store Analysis</h1>

In [44]:
# Store Performance Analysis:
top_10_stores_ir = merged_df.groupby('store_id')['ir'].sum().sort_values(ascending=False).head(10)
bottom_10_stores_isu = merged_df.groupby('store_id')['iSU'].sum().sort_values().head(10)
store_performance_by_city = merged_df.groupby(['city', 'store_id'])[['ir', 'iSU']].sum().reset_index()




In [55]:
# Performance of stores by city
fig_store_performance_city = px.scatter(store_performance_by_city.groupby('city').agg({'ir': 'sum', 'iSU': 'sum'}).reset_index(),
                                       x='ir', y='iSU', color='city', size='ir',
                                       title='Store Performance by City (Total IR in Each City)')
fig_store_performance_city.show()


In [45]:
import plotly.express as px

# Store Performance Analysis:
# Top 10 stores in terms of Incremental Revenue (IR)
fig_top_stores_ir = px.bar(top_10_stores_ir, x=top_10_stores_ir.index, y='ir', title='Top 10 Stores by Incremental Revenue')
fig_top_stores_ir.show()

# Bottom 10 stores in terms of Incremental Sold Units (ISU)
fig_bottom_stores_isu = px.bar(bottom_10_stores_isu, x=bottom_10_stores_isu.index, y='iSU', title='Bottom 10 Stores by Incremental Sold Units')
fig_bottom_stores_isu.show()

# Performance of stores by city
fig_store_performance_city = px.scatter(store_performance_by_city, x='ir', y='iSU', color='city', size='ir',
                                        title='Store Performance by City')
fig_store_performance_city.show()



<h1 align="center">Exploratory Promotion type Analysis</h1>

In [56]:
# Promotion Type Analysis:

# Product and Category Analysis:

# Top 2 promotion types by Incremental Revenue
fig_top_promo_types_ir = px.bar(top_2_promo_types_ir, x=top_2_promo_types_ir.index, y='ir', title='Top 2 Promo Types by Incremental Revenue')
fig_top_promo_types_ir.show()

# Bottom 2 promotion types by Incremental Sold Units
fig_bottom_promo_types_isu = px.bar(bottom_2_promo_types_isu, x=bottom_2_promo_types_isu.index, y='iSU', title='Bottom 2 Promo Types by Incremental Sold Units')
fig_bottom_promo_types_isu.show()


<h1 align="center">Exploratory Categories Analysis</h1>

In [57]:

# Product and Category Analysis:

top_categories_lift = merged_df.groupby('category')['iSU'].sum().sort_values(ascending=False)
# Top categories by Incremental Sold Units
fig_top_categories_lift = px.bar(top_categories_lift, x=top_categories_lift.index, y='iSU', title='Top Categories by Incremental Sold Units')
fig_top_categories_lift.show()


<h1 align="center">Query 1</h1>

In [47]:
#query 1

high_value_discounted_products = merged_df[(merged_df['base_price'] > 500) & (merged_df['promo_type'] == 'BOGOF')]
print("Query 1: High-value products with base price > 500 and BOGOF promo")
print(high_value_discounted_products[['product_code', 'product_name', 'base_price', 'promo_type']])
print()

# Visualization 1: Scatter plot of High-value products
fig_high_value = px.scatter(high_value_discounted_products, x='product_name', y='base_price',
                            color='promo_type', title='High-value products with BOGOF promo')
fig_high_value.show()


Query 1: High-value products with base price > 500 and BOGOF promo
     product_code                    product_name  base_price promo_type
6             P08       Atliq_Double_Bedsheet_set        1190      BOGOF
7             P08       Atliq_Double_Bedsheet_set        1190      BOGOF
22            P14  Atliq_waterproof_Immersion_Rod        1020      BOGOF
23            P14  Atliq_waterproof_Immersion_Rod        1020      BOGOF
36            P08       Atliq_Double_Bedsheet_set        1190      BOGOF
...           ...                             ...         ...        ...
1463          P14  Atliq_waterproof_Immersion_Rod        1020      BOGOF
1476          P08       Atliq_Double_Bedsheet_set        1190      BOGOF
1477          P08       Atliq_Double_Bedsheet_set        1190      BOGOF
1492          P14  Atliq_waterproof_Immersion_Rod        1020      BOGOF
1493          P14  Atliq_waterproof_Immersion_Rod        1020      BOGOF

[200 rows x 4 columns]



<h1 align="center">Query 2</h1>

In [48]:
# Query 2
store_counts_by_city = dim_stores['city'].value_counts().reset_index()
store_counts_by_city.columns = ['city', 'store_count']
print("Query 2: Overview of the number of stores in each city")
print(store_counts_by_city)
print()

# Visualization 2: Bar chart of store counts by city
fig_store_counts = px.bar(store_counts_by_city, x='city', y='store_count',
                          title='Number of Stores in Each City')
fig_store_counts.show()

Query 2: Overview of the number of stores in each city
            city  store_count
0      Bengaluru           10
1        Chennai            8
2      Hyderabad            7
3  Visakhapatnam            5
4     Coimbatore            5
5        Madurai            4
6         Mysuru            4
7      Mangalore            3
8     Trivandrum            2
9     Vijayawada            2



<h1 align="center">Query 3</h1>

In [49]:
campaign_revenue_report = merged_df.groupby('campaign_name').agg({
    'revenue_before_promotion': 'sum',
    'revenue_after_promotion': 'sum'
}).reset_index()
campaign_revenue_report['revenue_before_promotion'] /= 1e6  # Convert to millions
campaign_revenue_report['revenue_after_promotion'] /= 1e6  # Convert to millions
print("Query 3: Campaign-wise Revenue Report (in millions)")
print(campaign_revenue_report)
print()

# Visualization 3: Bar chart of revenue before and after promotion by campaign
fig_campaign_revenue = px.bar(campaign_revenue_report, x='campaign_name',
                              y=['revenue_before_promotion', 'revenue_after_promotion'],
                              title='Campaign-wise Revenue Report (in millions)',
                              labels={'value': 'Revenue (in millions)'})
fig_campaign_revenue.show()

Query 3: Campaign-wise Revenue Report (in millions)
  campaign_name  revenue_before_promotion  revenue_after_promotion
0        Diwali                 82.573759               171.459777
1     Sankranti                 58.127429               124.146867



<h1 align="center">Query 4</h1>

In [50]:
# Modified Query 4
diwali_category_isu = merged_df[merged_df['campaign_name'] == 'Diwali'].groupby('category').agg({
    'quantity_sold(before_promo)': 'sum',
    'new_quantity': 'sum'
}).reset_index()

print(diwali_category_isu)

diwali_category_isu['isu_percentage'] = ((diwali_category_isu['new_quantity'] - diwali_category_isu['quantity_sold(before_promo)']) / diwali_category_isu['quantity_sold(before_promo)']) * 100
diwali_category_isu['rank'] = diwali_category_isu['isu_percentage'].rank(ascending=False).astype(int)

diwali_category_isu = diwali_category_isu.sort_values(by='rank').reset_index(drop=True)

print("Modified Query 4: Incremental Sold Quantity Percentage (ISU%) for Each Category during Diwali Campaign")
print(diwali_category_isu[['category', 'isu_percentage', 'rank']])



            category  quantity_sold(before_promo)  new_quantity
0             Combo1                        16791         50769
1  Grocery & Staples                        58129         68620
2    Home Appliances                         5230         36006
3          Home Care                        13326         40396
4      Personal Care                        16843         22074
Modified Query 4: Incremental Sold Quantity Percentage (ISU%) for Each Category during Diwali Campaign
            category  isu_percentage  rank
0    Home Appliances      588.451243     1
1          Home Care      203.136725     2
2             Combo1      202.358406     3
3      Personal Care       31.057413     4
4  Grocery & Staples       18.047790     5


<h1 align="center">Query 5</h1>

In [51]:
# Modified Query 5
campaign_top5_products_ir = merged_df.groupby(['campaign_name', 'product_name']).agg({
    'ir': 'sum',
    'revenue_before_promotion': 'sum'
}).reset_index()

campaign_top5_products_ir['ir_percentage'] = (campaign_top5_products_ir['ir'] / campaign_top5_products_ir['revenue_before_promotion']) * 100
campaign_top5_products_ir['rank'] = campaign_top5_products_ir.groupby('campaign_name')['ir_percentage'].rank(ascending=False).astype(int)

campaign_top5_products_ir = campaign_top5_products_ir[campaign_top5_products_ir['rank'] <= 5]

print("Modified Query 5: Campaign-wise Top 5 Products by Incremental Revenue Percentage")
print(campaign_top5_products_ir[['campaign_name', 'product_name', 'ir_percentage', 'rank']])
print()

# Visualization 5: Bar chart of IR% for Top 5 products by campaign
fig_campaign_top5_ir = px.bar(campaign_top5_products_ir.sort_values(by='rank'), 
                              x='product_name', y='ir_percentage', color='campaign_name',
                              title='Campaign-wise Top 5 Products by Incremental Revenue Percentage',
                              labels={'ir_percentage': 'IR Percentage'})
fig_campaign_top5_ir.show()


Modified Query 5: Campaign-wise Top 5 Products by Incremental Revenue Percentage
   campaign_name                          product_name  ir_percentage  rank
2         Diwali                        Atliq_Curtains     243.805970     2
4         Diwali             Atliq_Double_Bedsheet_set     242.667928     4
7         Diwali           Atliq_High_Glo_15W_LED_Bulb     243.794712     3
8         Diwali  Atliq_Home_Essential_8_Product_Combo     151.965339     5
14        Diwali        Atliq_waterproof_Immersion_Rod     244.913151     1
19     Sankranti             Atliq_Double_Bedsheet_set     274.054572     5
20     Sankranti          Atliq_Farm_Chakki_Atta (1KG)     275.128971     4
22     Sankranti           Atliq_High_Glo_15W_LED_Bulb     275.248509     3
28     Sankranti               Atliq_Suflower_Oil (1L)     276.360952     1
29     Sankranti        Atliq_waterproof_Immersion_Rod     275.814058     2

